# Product Category Classification DL

In [1]:
import os
import pandas as pd
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm
import torch
import numpy as np

## Summary
- <a href='#1'>1. Data Loading</a>
- <a href='#2'>2. Data Preprocessing</a>
- <a href='#3'>3. Data Loader</a>
- <a href='#4'>4. Model</a>
- <a href='#5'>5. Training</a>
- <a href='#6'>6. Evaluation</a>
- <a href='#7'>7. Model Saving/Loading</a>

# <a id='1'>1. Data Loading</a>

In [2]:
FILE_PATH = "../../data/testset_C.csv" # File Path where the training files are located

In [3]:
df_train_preprocessing = pd.read_csv(FILE_PATH, sep=';')

# <a id='2'>2. Data Preprocessing</a>

In [4]:
df_train_preprocessing.loc[:,'main_text'] = df_train_preprocessing.main_text.astype('str')
df_train_preprocessing.loc[:,'add_text'] = df_train_preprocessing.add_text.astype('str')
df_train_preprocessing.loc[:,'manufacturer'] = df_train_preprocessing.manufacturer.astype('str')
df_train_preprocessing['all_text'] = df_train_preprocessing['main_text']+ ' ' +df_train_preprocessing['add_text'] + ' ' + df_train_preprocessing['manufacturer']

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'BICYCLES':0,
          'WASHINGMACHINES':1,
          'USB MEMORY':2,
          'CONTACT LENSES':3
          }

# <a id='3'>3. Data Loader</a>

In [6]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['productgroup']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['all_text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

# <a id='4'>4. Model</a>

In [7]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 4)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer


# <a id='5'>5. Training</a>

In [8]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=16, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=16)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                 

# <a id='6'>6. Evaluation</a>

In [9]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=16)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():
        y_pred_list = []
        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)
                
              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [10]:
df_train, df_val = np.split(df_train_preprocessing.sample(frac=1, random_state=42), 
                                     [int(.8*len(df_train_preprocessing))])

print(len(df_train),len(df_val))

6400 1600


In [ ]:
EPOCHS = 2  # epochs
model = BertClassifier()
LR = 1e-6 # Learning Rate
              
train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  5%|▌         | 20/400 [21:55<7:09:12, 67.77s/it]

In [ ]:
evaluate(model, df_test_preprocessing)

# <a id='7'>7. Model Saving</a>

In [ ]:
torch.save(model.state_dict(), 'gfk_model_bert.pt') # this model has arround 414 mb

In [ ]:
## Load the model
model = BertClassifier()
model.load_state_dict(torch.load('gfk_model_bert.pt'))
model.eval()

# <a id='4'>4 Conclusion</a>

Here I trained a DL model to detect 4 categories I was not able to finish due to the fact that the trainig takes about 6 hours to compute. 